<a href="https://colab.research.google.com/github/podschwadt/teaching/blob/master/intro/tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow 1

This a demonstration of how to use the Tenserflow 1 graph. How to build it and how to execute the graph. For more indepth info on the graph in TF 1 check here: https://github.com/tensorflow/docs/blob/master/site/en/r1/guide/graphs.md 

In short. TF 1 handles its program in two steps. 


1.   Defining the graph and all the operations. Varialbe input into the graph is defined by empty placeholders.
2.   Executing the graph using a session. This allows us to get results and feed data into the place holders.

Creating a GAN using Tensorflow.

Using two networks, a generator **G** and discrimantor **D** we want to create fake instances from the MNIST data set. 

**G** takes some noise $z$ and creates a fake from it.

**D** trys to distinguish between fakes created by **G** and real samples from 
the training data.

The classic GAN loss is defined as:
$$
\min_G \max_D V(D, G)=
\mathbb{E}_{x\sim p_{data}(x)}[\log D(x)]
+ \mathbb{E}_{z\sim p_z(z)}[\log(1 - D(G(z)))]
$$


Define a number of hyperparameters

In [0]:
import tensorflow as tf
import numpy as np


BATCH_SIZE = 32
LEARNING_RATE = 0.1
NOISE_LENGTH = 10
EPOCHS = 10

Set up plotting 

In [0]:
%matplotlib inline 
import matplotlib.pyplot as plt

def show_image( img ):
    plt.imshow( img.reshape( 28, 28 ), cmap="gray_r" )
    plt.axis( 'off' )
    plt.show( )

Load the data

In [0]:
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.astype('float32')
x_train /= 255
x_train = x_train.reshape( x_train.shape[ 0 ], -1 )

print( x_train.shape )

Define the Discriminator and Generator graph. Due to the way how Tensorflow 1 handles variables we are building a convience function to select all varibels that have a specific naming scheme.

Reusing variables means that we do not create new subgraphs every time the function is called but that we reuse what we created perviously.

In [0]:
# helper function to retrieve the variables with a specific name
def variables_with_name( name ):
    return [ x for x in tf.trainable_variables( ) if name in x.name ]

def Disctriminator( inputs ):
    with tf.variable_scope( '', reuse=tf.AUTO_REUSE ) as scope:
        inputs = tf.layers.dense( name='Disc.1', units=256, activation=tf.nn.relu, inputs=inputs )
        inputs = tf.layers.dense( name='Disc.2', units=1, activation=tf.nn.sigmoid, inputs=inputs )

        return inputs

def Generator( noise ):
    with tf.variable_scope( '', reuse=tf.AUTO_REUSE ) as scope:
        print( noise.shape )
        output = tf.layers.dense( name='Gen.1', units=256, activation=tf.nn.relu, inputs=noise )
        output = tf.layers.dense( name='Gen.2', units=x_train.shape[ 1 ], activation=tf.nn.sigmoid,
                                  inputs=output )
        return output


Defining input place holders for both the noise and real data.

In [0]:
real = tf.placeholder( tf.float32, shape=[ BATCH_SIZE, x_train.shape[ 1 ] ] )
noise_input = tf.placeholder( tf.float32, shape=[ BATCH_SIZE, NOISE_LENGTH ] )


Next we set up the graph that calcualtes the loss for the Generator and Discriminator as well as the optimizers to minimize that loss.

In [0]:

with tf.variable_scope( '', reuse=tf.AUTO_REUSE ) as scope:

    # run and update generator
    fake = Generator( noise_input )
    gen_los = tf.reduce_mean( tf.log( Disctriminator( fake ) ) )
    gen_params = variables_with_name( 'Gen' )
    gen_trainer = tf.train.AdamOptimizer( learning_rate=LEARNING_RATE ).minimize( gen_los, var_list=gen_params )
    
    # run and update discriminator
    fake_pred = Disctriminator( fake )
    real_pred = Disctriminator( real )

    disc_loss = tf.reduce_mean( -1. * tf.log( 1. - real_pred ) - tf.log( fake_pred ) )
    disc_params = variables_with_name( 'Disc' )
    disc_trainer = tf.train.AdamOptimizer( learning_rate=LEARNING_RATE ).minimize( disc_loss, var_list=disc_params )


Now we can use a session to exectute the parts of the graphs that perform training. To this we use the run `methode`. It takes a list or a single what is called fetches. This means graph elements that we want to get the output for. And it needs a `feed_dict`. The `feed_dict` is a dictornay that contains values for all the placeholders that are required to exectute that graph the `fetches` depend on.

In [0]:
    # training loop
    session = tf.Session( )
    session.run( tf.global_variables_initializer( ) )
    for epoch in range( EPOCHS ):
        np.random.shuffle( x_train )
        num_batches = x_train.shape[ 0 ] // BATCH_SIZE
        for i in range( num_batches ):
            batch = x_train[ i * BATCH_SIZE : (i+1) * BATCH_SIZE ]
            # print( "batch: ", batch.shape )
            # noise z
            z = np.random.normal( size=noise_input.shape )
            session.run( [ disc_loss, disc_trainer ], feed_dict={ real: batch, noise_input: z } )
            session.run( [ gen_los, gen_trainer ], feed_dict={noise_input: z} )
        print ( "epoch: ", epoch )

## Create some fakes

Feed a noise vector to the generator and print the results. It might not looks line anything. This is ok simple GANs are quite finicky to train. You can try and change some of the hyperparameters and see who the results change.

## Solution:

In [0]:
print( noise_input.shape )

# noise z
z = np.random.normal( size=noise_input.shape )

result = session.run( fake , feed_dict={ noise_input: z }  )

for x in result:
  show_image( x )